In [ ]:
import os
import sys
import math
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


sys.path.append('../..')
import model.Baseline as baseline
import model.my_model as mymodel
import model.util_loss as util_ls
import model.util_dataloader as util_dl
import model.util_model as util_md
import data_process.util_data as util_dt


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("The model will be running on", device, "device")

# local
file_save_dir = r'C:\Users\29492\Desktop\server_data\main_REDD'
file_load_dir = r'D:\dataset\DataFrames\REDD'
fig_save_dir = r'C:\Users\29492\Desktop\server_data\plot'
# server
# file_save_dir = r'../../data'
# file_load_dir = r'../../data'

dm = util_md.DictManager(file_save_dir)
mwm = util_md.ModelWeightManager(file_save_dir)

In [ ]:
def averaged_seq(pred_2):
    n_rows, in_seq_l = pred_2.shape
    window_l_half = math.floor((in_seq_l-1)/2)
    pred_2_seq = []
    for i in range(n_rows):
        row_s = max(0, i-window_l_half)
        row_e = min(n_rows-1, i+window_l_half)
        tmp = 0
        num = 0
        for j in range(row_s, row_e+1):
            num +=1
            tmp += pred_2[j,i+window_l_half-j]
        pred_2_seq.append(tmp/num)
    pred_2_seq = np.array(pred_2_seq)
    return pred_2_seq

In [ ]:
plt.rcParams['font.family'] = 'Times new Roman'  # Use a serif font family
plt.rcParams['font.size'] = 14
%matplotlib qt

In [ ]:
dataframe_path = os.path.join(file_load_dir, r'house_1.csv')
df = pd.read_csv(dataframe_path)
df.set_index('time', inplace=True)
# df.describe(percentiles=[0.10,0.25,0.50,0.75,0.8,0.85,0.9,0.95])


house_1_cols = ['oven-3', 'oven-4', 'refrigerator-5', 'dishwaser-6', 'lighting-9', 'washer_dryer-10', 'microwave-11',
                'bathroom_gfi-12', 'electric_heat-13', 'stove-14', 'lighting-17', 'lighting-18', 'washer_dryer-19', 'washer_dryer-20']
filtered_columns = df[house_1_cols]
# filtered_columns.describe(percentiles=[0.10,0.25,0.50,0.6,0.75,0.8,0.85,0.9,0.95])


apps_name = filtered_columns.columns.to_list()
print(apps_name)


filtered_columns['sum'] = filtered_columns.sum(axis=1)
norm_data, min_val, max_val = util_dt.normalization_interval(filtered_columns.to_numpy(), 0, 1)
main = norm_data[:,-1]
apps = norm_data[:,:-1]
print(f'min: {min_val}; max: {max_val}')
print(np.min(main), np.max(main))
print(np.min(apps), np.max(apps))
util_dt.print_shape(main, apps)



In [ ]:
# on_threshold = [50, 50, 50, 50, 20, 50, 50, 50, 50, 50, 20, 20, 50, 50]
on_threshold = 30
sliding_window_len = 599


w_main, w_apps = util_dt.generate_window_samples(input_1=main, input_2=apps, window_size_1=sliding_window_len,
                                     window_size_2=1, offset=math.floor(sliding_window_len/2))
w_main = np.expand_dims(w_main, axis=1)
w_apps_sum = w_apps.sum(axis=-1)
w_apps_sum = np.expand_dims(w_apps_sum, axis=1)
w_apps_on = (w_apps>(on_threshold-min_val)/(max_val-min_val)).astype('int')

util_dt.print_shape(w_main, w_apps, w_apps_sum, w_apps_on)

In [ ]:
in_seq_l = sliding_window_len
out_dim = len(apps_name)
print(f'in_seq_l: {in_seq_l}; out_dim: {out_dim}')


apps_to_train = ['refrigerator-5', 'dishwaser-6', 'washer_dryer-10', 'microwave-11', 'bathroom_gfi-12']
for app in apps_to_train:
    print(apps_name.index(app), app)

# plot

In [ ]:
index_app = 2
appname = apps_name[index_app]
print(appname)
index_s = 0
index_e = 100000
time = np.arange(0, index_e-index_s)
time_hours = time * 8 / 3600

tmp_input = w_main[index_s:index_e]
tmp_sum = w_apps_sum[index_s:index_e, 0]*(max_val-min_val)+min_val
tmp_app = w_apps[index_s:index_e, index_app]*(max_val-min_val)+min_val

fig = plt.figure(figsize=(15, 15))
plt.plot(time, tmp_sum, c='green', label = 'Input', linewidth=0.5)
plt.plot(time, tmp_app, c='blue', label = 'Ground truth', linewidth=1.5)

In [ ]:
tmp_input = w_main[index_s:index_e]
tmp_sum = w_apps_sum[index_s:index_e, 0]*(max_val-min_val)+min_val
tmp_app = w_apps[index_s:index_e, index_app]*(max_val-min_val)+min_val

dataset = util_dl.VariableDataset(torch.from_numpy(tmp_input).to(torch.float32))
dataset.describe()
testset_599 = util_dl.just_to_DataLoader(dataset=dataset, batch_size=100, shuffle_flag=False)
dataset = util_dl.VariableDataset(torch.from_numpy(tmp_input[:,:,100:-100]).to(torch.float32))
dataset.describe()
testset_399 = util_dl.just_to_DataLoader(dataset=dataset, batch_size=100, shuffle_flag=False)

fig_size = (3,3)
color_GT = 'black'
label_GT = 'Ground truth'
width_GT = 0.5
color_Pred = 'blue'
label_Pred = 'Prediction'
width_Pred = 1
y_max = 2100
y_min = -100
y_span = 200
y_ticks = np.arange(y_min, y_max + y_span, y_span)

In [ ]:
fig0, ax0 = plt.subplots(figsize=fig_size)
ax0.plot(time_hours, tmp_sum, c='green', label = 'Input', linewidth=0.5)
ax0.plot(time_hours, tmp_app, c=color_GT, label = label_GT, linewidth=1.5)
ax0.legend()
# ax0.set_xlabel('Time (hour)')
# ax0.set_ylabel('Power')
fig0.savefig(os.path.join(fig_save_dir, f'main_GT_{appname}.png'), dpi=1000, bbox_inches='tight')




model_DAE = baseline.DAE(399)
mwm.load_model_weight(model_DAE, f'REDD_DAE-399_{appname}_0111')
pred_DAE = util_md.predict(device, model_DAE, testset_399)[0]*(max_val-min_val)+min_val
pred_DAE = np.squeeze(pred_DAE)

fig1, ax1 = plt.subplots(figsize=fig_size)
ax1.plot(time_hours, tmp_app, c=color_GT, label = label_GT, linewidth=width_GT)
# ax1.plot(time_hours, pred_DAE[:,300], c=color_Pred, label = label_Pred, linewidth=width_Pred)
ax1.plot(time_hours, averaged_seq(pred_DAE), c=color_Pred, label = label_Pred, linewidth=width_Pred)
ax1.set_ylim(y_min, y_max)
ax1.set_yticks(y_ticks)
# ax1.set_xlabel('Time (hour)')
# ax1.set_ylabel('Power')
ax1.legend()
fig1.savefig(os.path.join(fig_save_dir, f'main_DAE-399_{appname}.png'), dpi=1000, bbox_inches='tight')


model_S2S = baseline.Seq2Seq(399, 399)
mwm.load_model_weight(model_S2S, f'REDD_S2S-399_{appname}_0111')
pred_S2S = util_md.predict(device, model_S2S, testset_399)[0]*(max_val-min_val)+min_val
pred_S2S = averaged_seq(pred_S2S)

fig2, ax2 = plt.subplots(figsize=fig_size)
ax2.plot(time_hours, tmp_app, c=color_GT, label = label_GT, linewidth=width_GT)
ax2.plot(time_hours, pred_S2S, c=color_Pred, label = label_Pred, linewidth=width_Pred)
ax2.set_ylim(y_min, y_max)
ax2.set_yticks(y_ticks)
# ax2.set_xlabel('Time (hour)')
# ax2.set_ylabel('Power')
ax2.legend()
fig2.savefig(os.path.join(fig_save_dir, f'main_S2S-399_{appname}.png'), dpi=1000, bbox_inches='tight')


model_S2P = baseline.Seq2Point(in_seq_l)
mwm.load_model_weight(model_S2P, f'REDD_S2P_{appname}_1228')
pred_S2P = util_md.predict(device, model_S2P, testset_599)[0]*(max_val-min_val)+min_val

fig3, ax3 = plt.subplots(figsize=fig_size)
ax3.plot(time_hours, tmp_app, c=color_GT, label = label_GT, linewidth=width_GT)
ax3.plot(time_hours, pred_S2P, c=color_Pred, label = label_Pred, linewidth=width_Pred)
ax3.set_ylim(y_min, y_max)
ax3.set_yticks(y_ticks)
# ax3.set_xlabel('Time (hour)')
# ax3.set_ylabel('Power')
ax3.legend()
fig3.savefig(os.path.join(fig_save_dir, f'main_S2P_{appname}.png'), dpi=1000, bbox_inches='tight')


model_mark = mymodel.Model_final(in_seq_l, out_dim)
mwm.load_model_weight(model_mark, f'REDD_Final_all_1228-{2}')
pred_mark = util_md.predict(device, model_mark, testset_599)[1][:, index_app]*(max_val-min_val)+min_val

fig4, ax4 = plt.subplots(figsize=fig_size)
ax4.plot(time_hours, tmp_app, c=color_GT, label = label_GT, linewidth=width_GT)
ax4.plot(time_hours, pred_mark, c=color_Pred, label = label_Pred, linewidth=width_Pred)
ax4.set_ylim(y_min, y_max)
ax4.set_yticks(y_ticks)
# ax4.set_xlabel('Time (hour)')
# ax4.set_ylabel('Power')
ax4.legend()
fig4.savefig(os.path.join(fig_save_dir, f'main_mark_{appname}.png'), dpi=1000, bbox_inches='tight')

In [ ]:
model_DAE = baseline.DAE(in_seq_l)
mwm.load_model_weight(model_DAE, f'REDD_DAE_{appname}_1228')
pred_DAE = util_md.predict(device, model_DAE, testset)[0]*(max_val-min_val)+min_val
pred_DAE = np.squeeze(pred_DAE)

fig1, ax1 = plt.subplots(figsize=fig_size)
ax1.plot(time_hours, tmp_app, c=color_GT, label = label_GT, linewidth=width_GT)
# ax1.plot(time_hours, pred_DAE[:,300], c=color_Pred, label = label_Pred, linewidth=width_Pred)
ax1.plot(time_hours, pred_DAE[:,300], c=color_Pred, label = label_Pred, linewidth=width_Pred)
ax1.set_ylim(y_min, y_max)
ax1.set_yticks(y_ticks)
# ax1.set_xlabel('Time (hour)')
# ax1.set_ylabel('Power')
ax1.legend()
# fig1.savefig(os.path.join(fig_save_dir, f'main_DAE_{appname}.png'), dpi=1000, bbox_inches='tight')

In [ ]:
model_S2S = baseline.Seq2Seq(in_seq_l, in_seq_l)
mwm.load_model_weight(model_S2S, f'REDD_S2S_{appname}_1228')
pred_S2S = util_md.predict(device, model_S2S, testset)[0]*(max_val-min_val)+min_val

fig2, ax2 = plt.subplots(figsize=fig_size)
ax2.plot(time_hours, tmp_app, c=color_GT, label = label_GT, linewidth=width_GT)
ax2.plot(time_hours, pred_S2S[:,300], c=color_Pred, label = label_Pred, linewidth=width_Pred)
ax2.set_ylim(y_min, y_max)
ax2.set_yticks(y_ticks)
# ax2.set_xlabel('Time (hour)')
# ax2.set_ylabel('Power')
ax2.legend()